# Transkript Bot (Colab)
This notebook clones the repo, installs dependencies with uv, and runs the bot.

Note: a local Bot API server is started by default for files >20MB. You can disable it by typing 'off' when asked for BOT_API_BASE_URL.

In [ ]:
# Optional: mount Google Drive for persistent storage
# from google.colab import drive
# drive.mount('/content/drive')


In [ ]:
import os
from getpass import getpass

os.environ['BOT_TOKEN'] = getpass('BOT_TOKEN: ').strip()
os.environ['HF_TOKEN'] = getpass('HF_TOKEN (optional): ').strip()
os.environ['ROOT_ADMIN_IDS'] = input('ROOT_ADMIN_IDS (comma-separated): ').strip()
bot_api = input("BOT_API_BASE_URL (default http://localhost:8081, type 'off' to disable): ").strip()
if bot_api.lower() == 'off':
    os.environ.pop('BOT_API_BASE_URL', None)
elif bot_api:
    os.environ['BOT_API_BASE_URL'] = bot_api
else:
    os.environ['BOT_API_BASE_URL'] = 'http://localhost:8081'
# Optional: storage path if using Drive
# os.environ['STORAGE_PATH'] = '/content/drive/MyDrive/transkript/bot.db'
# Optional: force CPU backend
# os.environ['BACKEND_FORCE'] = 'faster'
# Optional: GitHub token for private repo (HTTPS)
# os.environ['GITHUB_TOKEN'] = getpass('GITHUB_TOKEN: ').strip()
# Optional: SSH private key for private repo
# os.environ['SSH_PRIVATE_KEY'] = getpass('SSH_PRIVATE_KEY: ').strip()


In [ ]:
import os
from pathlib import Path

# Option A: HTTPS + token (private repo)
# REPO_URL = 'https://github.com/OWNER/REPO.git'
# GITHUB_TOKEN = os.environ.get('GITHUB_TOKEN', '').strip()
# !git -c http.extraheader="Authorization: Bearer $GITHUB_TOKEN" clone $REPO_URL

# Option B: SSH deploy key (private repo)
# REPO_URL = 'git@github.com:OWNER/REPO.git'
# key = Path('/root/.ssh/id_ed25519')
# key.parent.mkdir(parents=True, exist_ok=True)
# key.write_text(os.environ.get('SSH_PRIVATE_KEY', ''))
# os.chmod(key, 0o600)
# !ssh-keyscan github.com >> /root/.ssh/known_hosts
# !git clone $REPO_URL

# Default: public repo
REPO_URL = 'https://github.com/USER/REPO.git'
repo_dir = REPO_URL.rstrip('/').split('/')[-1].replace('.git', '')

!pip -q install uv
!git clone $REPO_URL
os.chdir(repo_dir)
!uv venv
!uv sync
# Optional: install WhisperX for GPU diarization
# !uv pip install whisperx


In [ ]:
import os
import shutil
import socket
import subprocess
import time
from pathlib import Path

# Start local Bot API server for large files (>20MB)
if os.environ.get('BOT_API_BASE_URL'):
    bot_api_bin = shutil.which('telegram-bot-api')
    if not bot_api_bin:
        subprocess.run(['bash', '-lc', 'apt-get -qq update'], check=True)
        subprocess.run([
            'bash', '-lc',
            'apt-get -qq install -y git make g++ cmake gperf zlib1g-dev libssl-dev libreadline-dev liblz4-dev libzstd-dev libsqlite3-dev pkg-config'
        ], check=True)
        subprocess.run([
            'bash', '-lc',
            'git clone --depth 1 https://github.com/tdlib/telegram-bot-api.git /content/telegram-bot-api'
        ], check=True)
        subprocess.run([
            'bash', '-lc',
            'cd /content/telegram-bot-api && git submodule update --init --recursive'
        ], check=True)
        subprocess.run([
            'bash', '-lc',
            'cmake -S /content/telegram-bot-api -B /content/telegram-bot-api/build -DCMAKE_BUILD_TYPE=Release'
        ], check=True)
        subprocess.run([
            'bash', '-lc',
            'cmake --build /content/telegram-bot-api/build -j 2'
        ], check=True)
        bot_api_bin = '/content/telegram-bot-api/build/telegram-bot-api'

    os.makedirs('/content/telegram-bot-api-data', exist_ok=True)
    log = open('/content/telegram-bot-api.log', 'w')
    subprocess.Popen([
        bot_api_bin,
        '--local',
        '--http-port=8081',
        '--dir=/content/telegram-bot-api-data',
    ], stdout=log, stderr=log)

    for _ in range(40):
        sock = socket.socket()
        sock.settimeout(0.5)
        try:
            sock.connect(('127.0.0.1', 8081))
            sock.close()
            print('Local Bot API server started on http://localhost:8081')
            break
        except Exception:
            time.sleep(0.5)
        finally:
            sock.close()


In [ ]:
!uv run python -m transkript_bot.main
